## Imports

In [1]:
from src.toyencoder import CKKSToyEncoder
from src.encoder import CKKSEncoder
from src.scheme import CkksScheme
from numpy.polynomial import Polynomial
import numpy as np
from sympy import nextprime
from openfhe import *

## Try the toy encoder homomorphism

In [2]:
ckks_encoder = CKKSToyEncoder(8)       # So the maximum size vector is 4 (8/2)
poly1 = ckks_encoder.sigma_inverse([2+3j, 3-9j, 4-2j, 1+4j])
poly2 = ckks_encoder.sigma_inverse([1-1j, 5+5j, 6-6j, 9-7j])

print("Poly1: ", poly1)
print("Poly2: ", poly2)

poly_multi = poly1 * poly2
poly_sum = poly1 + poly2

print("Poly_Multi: ", poly_multi)
print("Poly_Sum: ", poly_sum)

real_multi = ckks_encoder.sigma(poly_multi)
real_sum = ckks_encoder.sigma(poly_sum)

print("Real_Multi: ", real_multi)
print("Real_Sum: ", real_sum)

Poly1:  (2.500000000000002-1.0000000000000007j) +
(0.7071067811865489+3.1819805153394625j)·x -
(1.499999999999999-0.4999999999999988j)·x² +
(2.1213203435596393-3.181980515339467j)·x³
Poly2:  (5.249999999999997-2.2500000000000013j) -
(3.181980515339465+2.8284271247461894j)·x +
(1.2499999999999976-1.7500000000000004j)·x² -
(2.8284271247461907+0.35355339059326985j)·x³
Poly_Multi:  (10.875000000000004-10.875000000000009j) +
(0.08838834764831383+11.225320151336422j)·x +
(1.374999999999992-11.750000000000014j)·x² +
(9.192388155425071-14.142135623730955j)·x³ -
(17.625000000000014+1.8749999999999751j)·x⁴ +
(1.5026019100213932-8.573669721886883j)·x⁵ -
(7.1249999999999805-8.25000000000002j)·x⁶
Poly_Sum:  (7.75-3.2500000000000018j) - (2.4748737341529163-0.35355339059327306j)·x -
(0.25000000000000133+1.2500000000000018j)·x² -
(0.7071067811865515+3.535533905932737j)·x³
Real_Multi:  [ 5. +1.j 60.-30.j 12.-36.j 37.+29.j]
Real_Sum:  [ 3.+2.j  8.-4.j 10.-8.j 10.-3.j]


## Try the CKKS encoder

In [8]:
print("Iniciou")
ckks_encoder = CKKSEncoder(4, scale=2**30)       # So the maximum size vector is 4 (8/2)
print("Gerou encoder")
c_vec1 = np.array([2, 3])
c_vec2 = np.array([1, 5])

poly1 = ckks_encoder.encode(c_vec1)
poly2 = ckks_encoder.encode(c_vec2)

print("Poly1: ", poly1)
print("Poly2: ", poly2)

poly_multi = poly1 * poly2 
poly_sum = poly1 + poly2


print("Poly_Multi: ", poly_multi)
print("Poly_Sum: ", poly_sum)

real_multi = ckks_encoder.decode(poly_multi/ckks_encoder.scale)
real_sum = ckks_encoder.decode(poly_sum)

print("Real_Multi: ", real_multi)
print("Real_Sum: ", real_sum)

Iniciou
Gerou encoder
Poly1:  2.14748365e+09 + (3.22122547e+09)·x + (2.14748365e+09)·x² +
(3.22122547e+09)·x³
Poly2:  1.07374182e+09 + (5.36870912e+09)·x + (1.07374182e+09)·x² +
(5.36870912e+09)·x³
Poly_Multi:  2.30584301e+18 + (1.49879796e+19)·x + (2.19055086e+19)·x² +
(2.99759591e+19)·x³ + (3.68934881e+19)·x⁴ + (1.49879796e+19)·x⁵ +
(1.72938226e+19)·x⁶
Poly_Sum:  3.22122547e+09 + (8.58993459e+09)·x + (3.22122547e+09)·x² +
(8.58993459e+09)·x³
Real_Multi:  [ 2. 13.]
Real_Sum:  [3. 8.]


### Encryption and Decryption

In [10]:
def print_vector(vector):
    for i in vector:
        print("-> ", i)

# Parameters
N = 4                       # Degree of the cyclotomic ring (must be power of 2)
p = 2**20                   # Primo usado no contexto de reliniarização (exemplo)
scale = 2**20               # Scale factor for encoding (decimal precision of 10 bits)
noise_scale = 0.1           # Scale of the noise added during encryption
qs = [2**20, 2**40, 2**60]  # Different moduli for the ciphertexts


# Instantiate the encoder
encoder = CKKSEncoder(N=N, scale=scale)

# Generate a random secret key
secret_key_coeffs = np.random.randint(0, 2, encoder.N)
secret_key = Polynomial(secret_key_coeffs)

# Create CKKS scheme instance
scheme = CkksScheme(encoder, secret_key, noise_scale, p=p, qs=qs)
z1 = np.array([1, 8])
z2 = np.array([2, 2])

# Encryption
c1 = scheme.encrypt(z1)
c2 = scheme.encrypt(z2)

c_add = scheme.add(c1, c2)
c_multi = scheme.mult(c1, c2)

# Decryption
z_add = scheme.decrypt(c_add)
z_multi = scheme.decrypt(c_multi)

print("Vec1:", z1)
print("Vec2:",z2)

print("Soma:", z1+z2)
print_vector(z_add)
print("Polynomial")
print_vector(c_add)

print("\nMulti: ", z1*z2)
print_vector(z_multi)
print("Polynomial")
print_vector(c_multi)

Vec1: [1 8]
Vec2: [2 2]
Soma: [ 3 10]
->  3.0000000461693532
->  10.000000052580534
Polynomial
->  3145728.04841208 + 10485760.05513469·x + 3145727.92157283·x² +
10485759.91670715·x³ - 0.2799737·x⁴ - 0.75044042·x⁵ - 0.6832453·x⁶ +
0.09588087·x⁷ + 0.79120795·x⁸ - 0.02318267·x⁹
->  0.15839979 + 0.5528039·x + 0.72540276·x² + 0.70939845·x³ - 0.09588087·x⁴ -
0.79120795·x⁵ + 0.02318267·x⁶

Multi:  [ 2 16]
->  2.0
->  18.0
Polynomial
->  2097152.0 + 18874368.0·x + 20971520.0·x² + 37748735.0·x³ + 35651584.0·x⁴ +
18874362.0·x⁵ + 16777207.0·x⁶ - 10.0·x⁷ - 8.0·x⁸ - 4.0·x⁹ + 1.0·x¹⁰ +
2.0·x¹¹ + -0.0·x¹² + -0.0·x¹³ + -0.0·x¹⁴ + -0.0·x¹⁵ + 0.0·x¹⁶ + 0.0·x¹⁷ +
-0.0·x¹⁸
->  0.0 + 2.0·x + 6.0·x² + 10.0·x³ + 12.0·x⁴ + 7.0·x⁵ + 5.0·x⁶ - 1.0·x⁷ -
2.0·x⁸ + 0.0·x⁹ + 0.0·x¹⁰ + 0.0·x¹¹ + 0.0·x¹² + -0.0·x¹³ + -0.0·x¹⁴ +
0.0·x¹⁵
